# Create SparkSession

In [2]:
from pyspark.sql import SQLContext, HiveContext,SparkSession



In [3]:
spark = SparkSession\
.builder\
.appName("test-hive-console")\
.master("local[*]")\
.config("spark.sql.warehouse.dir", "/user/hive/warehouse")\
.enableHiveSupport()\
.getOrCreate()

[Database(name='default', description='Default Hive database', locationUri='file:/user/hive/warehouse'),
 Database(name='demo', description='', locationUri='file:/user/hive/warehouse/demo.db')]

# show databases

In [4]:
spark.catalog.listDatabases()

[Database(name='default', description='Default Hive database', locationUri='file:/user/hive/warehouse'),
 Database(name='demo', description='', locationUri='file:/user/hive/warehouse/demo.db')]

# show tables

In [4]:
database = "demo"
spark.sql("use "+database)
spark.sql("show tables").show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
|    demo|           wgtraffic|      false|
|    demo|wgtraffic_event_a...|      false|
+--------+--------------------+-----------+



# write to local filesystem from hive table

In [5]:
database = "demo"
table = "wgtraffic"
sdf = spark.sql("select * from "+database+"."+table)
sdf.repartition(1).write.json("/tmp/data.json","overwrite")

# create table in database

In [ ]:
database = "demo"
spark.sql("create database if not exists "+database)
spark.sql("use "+database)
spark.sql("DROP TABLE IF EXISTS " + database + ".sample_tuhin")
spark.sql("CREATE TABLE IF NOT EXISTS " + database + ".guardduty_score (account_id STRING,service_count INT,severity INT, ipv4 STRING, created_at STRING, username STRING, pre_score DOUBLE,score DOUBLE)") 


In [ ]:
# from pyspark.sql.types import StringType

# x = spark.createDataFrame(["10","11","13"], StringType()).toDF("age")
# # x.saveAsTable('x_temp', mode='overwrite')
# x.write.mode("append").saveAsTable("haha.age_data")
# spark.sql("select * from haha.age_data").show()

# write to HIVE table from json data

In [5]:
database = "demo"
spark.sql("create database if not exists "+database)
table = "wgtraffic"
json_path = "/Users/tuhinsharma/Documents/sstech/demo/hive_data/demo_wgtraffic.json"
spark.sql("DROP TABLE IF EXISTS " + database + "."+table)
sdf = spark.read.json(json_path)
sdf.write.mode("overwrite").saveAsTable(database+"."+table)


In [6]:
sdf.printSchema()

root
 |-- alarm_id: string (nullable = true)
 |-- alarm_name: string (nullable = true)
 |-- alarm_type: string (nullable = true)
 |-- bucket: string (nullable = true)
 |-- cluster: string (nullable = true)
 |-- d: long (nullable = true)
 |-- direction: string (nullable = true)
 |-- disp: string (nullable = true)
 |-- dst_intf: string (nullable = true)
 |-- dst_ip: string (nullable = true)
 |-- dst_port: string (nullable = true)
 |-- dst_port_nat: string (nullable = true)
 |-- event_time: string (nullable = true)
 |-- info_1: string (nullable = true)
 |-- info_2: string (nullable = true)
 |-- info_3: string (nullable = true)
 |-- info_4: string (nullable = true)
 |-- info_5: string (nullable = true)
 |-- info_6: string (nullable = true)
 |-- log_type: string (nullable = true)
 |-- m: long (nullable = true)
 |-- msg: string (nullable = true)
 |-- original_string: string (nullable = true)
 |-- pckt_len: string (nullable = true)
 |-- policy: string (nullable = true)
 |-- pr_info: string (n

# show table content

In [15]:
database = "demo"
table = "wgtraffic"
sdf = spark.sql("select * from "+database+"."+table)
sdf.show()

+---------------+--------+------------+--------------------+--------------------+----------------+--------------------+--------------------+-------------------+----------------+--------+---+------------+--------------------+--------+--------------------+-------+---+---------------+--------+--------------------+-------+---+----------+-------------+----------+-------------+--------+------------+------+-------------+---+-------------+----+
|         dst_ip|dst_port|dst_port_nat|          event_time|              info_1|          info_2|              info_3|              info_4|             info_5|          info_6|log_type|  m|         msg|     original_string|pckt_len|              policy|pr_info|pri|processing_dttm|protocol|           proxy_act| raw_id| rc|       sid|           sn|  src_intf|       src_ip|src_port|src_port_nat|tag_id|    timestamp|ttl|unknown_field|   y|
+---------------+--------+------------+--------------------+--------------------+----------------+--------------------

In [17]:
print(",".join(sdf.columns[0:]))
print(sdf.columns)

dst_ip,dst_port,dst_port_nat,event_time,info_1,info_2,info_3,info_4,info_5,info_6,log_type,m,msg,original_string,pckt_len,policy,pr_info,pri,processing_dttm,protocol,proxy_act,raw_id,rc,sid,sn,src_intf,src_ip,src_port,src_port_nat,tag_id,timestamp,ttl,unknown_field,y
['dst_ip', 'dst_port', 'dst_port_nat', 'event_time', 'info_1', 'info_2', 'info_3', 'info_4', 'info_5', 'info_6', 'log_type', 'm', 'msg', 'original_string', 'pckt_len', 'policy', 'pr_info', 'pri', 'processing_dttm', 'protocol', 'proxy_act', 'raw_id', 'rc', 'sid', 'sn', 'src_intf', 'src_ip', 'src_port', 'src_port_nat', 'tag_id', 'timestamp', 'ttl', 'unknown_field', 'y']


In [7]:
sdf.select('tag_id', 'timestamp', 'ttl', 'unknown_field', 'y').describe().show()

+-------+------+--------------------+-----+-------------+------+
|summary|tag_id|           timestamp|  ttl|unknown_field|     y|
+-------+------+--------------------+-----+-------------+------+
|  count| 30000|               30000|30000|        30000| 30000|
|   mean|1007.0|1.528798961635733...|  0.0|         null|2018.0|
| stddev|   0.0|   7557885.973675217|  0.0|         null|   0.0|
|    min|  1007|       1528783202000|    0|             |  2018|
|    max|  1007|       1528807040000|    0|             |  2018|
+-------+------+--------------------+-----+-------------+------+



In [9]:
sdf.select("dst_port", "info_2", "info_1", "src_port").show()

+--------+----------------+--------------------+--------+
|dst_port|          info_2|              info_1|src_port|
+--------+----------------+--------------------+--------+
|      80|sent_bytes=11357|rcvd_bytes=137046...|   53093|
|     443|sent_bytes=18088|rcvd_bytes=50624;...|   31906|
|      80| sent_bytes=8527|rcvd_bytes=145977...|   46977|
|      80|sent_bytes=10523|rcvd_bytes=77019;...|   46704|
|     443| sent_bytes=5596|rcvd_bytes=82784;...|   44093|
|     443|sent_bytes=12011|rcvd_bytes=85192;...|   49852|
|     443|sent_bytes=12191|rcvd_bytes=113427...|   55796|
|     443|sent_bytes=15307|rcvd_bytes=127082...|   60972|
|      80|sent_bytes=11849|rcvd_bytes=104144...|   49392|
|     443|sent_bytes=10616|rcvd_bytes=67529;...|   14163|
|      80| sent_bytes=8180|rcvd_bytes=131419...|   32583|
|     443| sent_bytes=7565|rcvd_bytes=94448;...|   18043|
|     443|sent_bytes=12371|rcvd_bytes=133546...|   36170|
|     443|sent_bytes=14702|rcvd_bytes=153977...|   60333|
|      80|sent